<a href="https://colab.research.google.com/github/Ramkuchana/LLMs/blob/master/Fine-tuning%20a%20Representation%20Model%20for%20Binary%20Sentiment%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning a Representation Model for Binary Sentiment Classification

In [ ]:
# %%capture
!pip install datasets>=2.18.0 transformers>=4.38.2 sentence-transformers>=2.5.1 setfit>=1.0.3 accelerate>=0.27.2 seqeval>=1.2.2

This project involves:

* Fine-tuning the pre-trained BERT model, specifically bert-base-cased, along with the classification head as a single architecture for movie review sentiment classification.

* Reducing fine-tuning training time by freezing some layers while attempting to maintain nearly the same performance as a fully fine-tuned bert-base-cased model.

## IMDb dataset

We will use the IMDb dataset from Hugging Face's datasets library to fine-tune our model for binary sentiment classification. The dataset consists of movie reviews labeled as either positive or negative. The original dataset has balanced train and test datasets, each containing 25,000 labeled samples (reviews) and an additional 50,000 unlabeled samples for unsupervised learning.

However, we will only use a subset of the IMDb dataset to reduce the computational requirements. (as training the original full dataset alone on Google Colab's T4 GPU previously took me approximately 40 minutes)

let's first load and explore the original IMDb dataset.

In [ ]:
from datasets import load_dataset, concatenate_datasets

# Load the IMDb dataset
imdb_dataset = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# Print the new imdb dataset

print(imdb_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [ ]:
# Inspecting the dataset structure

print(imdb_dataset["train"].features)

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}


In [ ]:
# Checking the first review to see what it looks like

print(imdb_dataset["train"][0]["text"])

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

In [ ]:
# Checking the label of the first review

print(imdb_dataset["train"][0]["label"])

0


## Creating a Smaller Dataset



We will create a new IMDb dataset containing a balanced training set with 8,000 samples (reviews) and a test set with 2,000 samples from the original IMDb dataset.



In [ ]:
# Define a function to balance the dataset
def balance_dataset(dataset, label_col, num_samples):
    # Filter positive and negative examples
    positive_samples = dataset.filter(lambda example: example[label_col] == 1)
    negative_samples = dataset.filter(lambda example: example[label_col] == 0)

    # Subsample both to the desired number
    positive_samples = positive_samples.shuffle(seed=42).select(range(num_samples // 2))
    negative_samples = negative_samples.shuffle(seed=42).select(range(num_samples // 2))

    # Concatenate positive and negative examples to form a balanced dataset
    balanced_dataset = concatenate_datasets([positive_samples, negative_samples]).shuffle(seed=42)

    return balanced_dataset

In [ ]:
# Create a balanced train and test dataset
train_data = balance_dataset(imdb_dataset["train"], "label", 8000)
test_data = balance_dataset(imdb_dataset["test"], "label", 2000)

# Checking the datasets
print(f"train_data:\n {train_data}", end='\n\n')
print(f"test_data:\n {test_data}")

train_data:
 Dataset({
    features: ['text', 'label'],
    num_rows: 8000
})

test_data:
 Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})


# Fine-tuning the bert-base-cased model with classifier


We will use the bert-base-cased model, a variant of the pre-trained BERT model, for our binary sentiment classification task. This model has been pre-trained on a large corpus of text for tasks like masked language modeling and next sentence prediction, but it is not specifically trained for sentiment classification. Therefore, fine-tuning the model makes it feasible to use it for this specific classification task, which in our case is movie review sentiment classification.


We will fine-tune the bert-base-cased model along with the classification head (aka classifier) as a single architecture. During training, the weights of both the pre-trained BERT layers and the classification layer are updated using backpropagation. This fine-tuning process adapts the general language understanding of BERT to the specific task of sentiment classification by learning from the IMDb dataset.


### Loading the Model and Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding

# Load Model and Tokenizer
model_id = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Pad to the longest sequence in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


The AutoModelForSequenceClassification class adds a classification head on top of the BERT model. This head is typically a simple fully connected (dense) layer, which takes the output from BERT and maps it to the desired number of classes. In this case, there are 2 classes: positive (1) or negative (0). This is why we pass the argument num_labels=2.

The DataCollatorWithPadding ensures that all sequences in a batch are padded to the same length during training, making it easier to process batches of text data.


### Tokenizing the data.

we will define preprocess_function(), which uses the tokenizer to tokenize text input data. The truncation=True option makes sure that sequences exceeding the model's maximum token length are truncated.

In [ ]:
# Define preprocessing function
def preprocess_function(examples):
   """Tokenize input data"""
   return tokenizer(examples["text"], truncation=True)

# # Tokenize train and test data
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

## Define F1 Score Evaluation Metric

We will define a custom compute_metrics() function that will be later passed as an argument to the Trainer class for evaluating the model's performance. In this case, we evaluate our model using the F1 score, which is calculated using the evaluate library's pre-loaded F1 metric and returned as a dictionary.

In [ ]:
import numpy as np
import evaluate


def compute_metrics(eval_pred):

    """Calculate F1 score"""

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    load_f1 = evaluate.load("f1")
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]

    return {"f1": f1}

## Train and Evaluate the model

Now, we define the hyperparameters that we want to tune in the TrainingArguments class:

* learning_rate=2e-5: The learning rate used for updating model parameters.
* per_device_train_batch_size=16 and per_device_eval_batch_size=16: Batch size during training and evaluation.
* num_train_epochs=1: The number of epochs (complete passes through the dataset).
* weight_decay=0.01: A regularization term to prevent overfitting.
* save_strategy="epoch": The model will be saved at the end of each epoch.
* report_to="none": Disables reporting to external logging services.

In [ ]:
from transformers import TrainingArguments, Trainer

# Training arguments for parameter tuning
training_args = TrainingArguments(
   "model",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=1,
   weight_decay=0.01,
   save_strategy="epoch",
   report_to="none"
)

# Trainer which executes the training process
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


We will perform model training and evaluation using the Trainer class.

In [ ]:
trainer.train()

Step,Training Loss
500,0.311900


TrainOutput(global_step=500, training_loss=0.31193878173828127, metrics={'train_runtime': 742.959, 'train_samples_per_second': 10.768, 'train_steps_per_second': 0.673, 'total_flos': 2084785113806400.0, 'train_loss': 0.31193878173828127, 'epoch': 1.0})

The output of trainer.evaluate() will be a dictionary containing default metrics, along with the F1 score metric that we defined as part of the compute_metrics() function (since F1 was specified as the custom evaluation metric).

In [ ]:
trainer.evaluate()

{'eval_loss': 0.19807493686676025,
 'eval_f1': 0.9257425742574258,
 'eval_runtime': 59.8717,
 'eval_samples_per_second': 33.405,
 'eval_steps_per_second': 2.088,
 'epoch': 1.0}

From the results, we can observe that the training time for fine-tuning is approximately 742.95 seconds, and we achieved an F1 score of 0.925.

# Freezing some layers in the main BERT and fine-tuning

In the previous section, the fine-tuning process took about 15 minutes. To reduce training time, we can freeze certain layers in the model and only fine-tune specific layers while maintaining a reasonable performance.


In [ ]:
# Load Model and Tokenizer

model_id = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Note that we have reloaded the model again instead of using the previously fine-tuned model because we do not want to freeze layers in an already fine-tuned model. Doing so would mean training a model that has already been fine-tuned, which would defeat the purpose of what we are trying to achieve.

Now, let's print out the layers of the model.

In [ ]:
# Print layer names
for name, param in model.named_parameters():
    print(name)

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

We can see that we have 12 (0-11) encoder blocks consisting of attention heads, dense networks, and layer normalization.

So, let's freeze encoder blocks (0-9) and only allow two encoder blocks, along with the classification head, to be trainable. This reduces computational power by only updating part of the pre-trained model.

In [ ]:
# Encoder block 10 starts at index 165
# So we freeze everything before that block using the index
for index, (name, param) in enumerate(model.named_parameters()):
    if index < 165:
        param.requires_grad = False


In [ ]:
# Checking whether the model was correctly updated
for index, (name, param) in enumerate(model.named_parameters()):
     print(f"Parameter: {index}{name} ----- {param.requires_grad}")

Parameter: 0bert.embeddings.word_embeddings.weight ----- False
Parameter: 1bert.embeddings.position_embeddings.weight ----- False
Parameter: 2bert.embeddings.token_type_embeddings.weight ----- False
Parameter: 3bert.embeddings.LayerNorm.weight ----- False
Parameter: 4bert.embeddings.LayerNorm.bias ----- False
Parameter: 5bert.encoder.layer.0.attention.self.query.weight ----- False
Parameter: 6bert.encoder.layer.0.attention.self.query.bias ----- False
Parameter: 7bert.encoder.layer.0.attention.self.key.weight ----- False
Parameter: 8bert.encoder.layer.0.attention.self.key.bias ----- False
Parameter: 9bert.encoder.layer.0.attention.self.value.weight ----- False
Parameter: 10bert.encoder.layer.0.attention.self.value.bias ----- False
Parameter: 11bert.encoder.layer.0.attention.output.dense.weight ----- False
Parameter: 12bert.encoder.layer.0.attention.output.dense.bias ----- False
Parameter: 13bert.encoder.layer.0.attention.output.LayerNorm.weight ----- False
Parameter: 14bert.encoder.laye

## Train and Evalute the partially frozen layer model

In [ ]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


trainer.train()


Step,Training Loss
500,0.389800


TrainOutput(global_step=500, training_loss=0.389836181640625, metrics={'train_runtime': 322.8201, 'train_samples_per_second': 24.782, 'train_steps_per_second': 1.549, 'total_flos': 2084785113806400.0, 'train_loss': 0.389836181640625, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.244190514087677,
 'eval_f1': 0.9073610415623435,
 'eval_runtime': 59.2922,
 'eval_samples_per_second': 33.731,
 'eval_steps_per_second': 2.108,
 'epoch': 1.0}

We observed that the training time was reduced to 322.82 seconds, more than half the time of the fully fine-tuned model, while maintaining good performance with an F1 score of 0.90



# Freezing all layers in the main BERT and fine-tuning

For experimentation, we will now freeze all the layers in the BERT model, allowing only the classification head to be trainable. We will then evaluate how the model performs.

In [ ]:
# Load Model and Tokenizer

model_id = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
for name, param in model.named_parameters():

     # Trainable classification head
     if name.startswith("classifier"):
        param.requires_grad = True

      # Freeze everything else
     else:
        param.requires_grad = False

In [ ]:
# We can check whether the model was correctly updated

for name, param in model.named_parameters():
     print(f"Parameter: {name} ----- {param.requires_grad}")

Parameter: bert.embeddings.word_embeddings.weight ----- False
Parameter: bert.embeddings.position_embeddings.weight ----- False
Parameter: bert.embeddings.token_type_embeddings.weight ----- False
Parameter: bert.embeddings.LayerNorm.weight ----- False
Parameter: bert.embeddings.LayerNorm.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.query.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.query.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.key.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.key.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.value.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.value.bias ----- False
Parameter: bert.encoder.layer.0.attention.output.dense.weight ----- False
Parameter: bert.encoder.layer.0.attention.output.dense.bias ----- False
Parameter: bert.encoder.layer.0.attention.output.LayerNorm.weight ----- False
Parameter: bert.encoder.layer.0.attention.output

## Train and Evalute the completely frozen BERT model

In [ ]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


trainer.train()

Step,Training Loss
500,0.696100


TrainOutput(global_step=500, training_loss=0.6961300659179688, metrics={'train_runtime': 249.3582, 'train_samples_per_second': 32.082, 'train_steps_per_second': 2.005, 'total_flos': 2084785113806400.0, 'train_loss': 0.6961300659179688, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6841261982917786,
 'eval_f1': 0.5947006869479883,
 'eval_runtime': 59.5484,
 'eval_samples_per_second': 33.586,
 'eval_steps_per_second': 2.099,
 'epoch': 1.0}

As expected, the training time is the shortest (249.35 seconds) in this scenario, but the model's F1 score is only 0.59, the worst performance among all the experiments, as no fine-tuning was done in the main BERT layers.

# Conclusion

In this project, we demonstrated:

Fine-tuning BERT for movie review sentiment classification.

Reducing training time with only a slight drop in performance by selectively fine-tuning only part of the model.


